In [1]:
using CRCBS
using LightGraphs, MetaGraphs
G = initialize_full_grid_graph_CT()
starts = [1,5,8]
goals = [4,1,2]
lambda = 0.5
epsilon = 0.1
t_delay = 0.5
mapf = MAPF(G,starts,goals,lambda,epsilon,t_delay)


┌ Info: Recompiling stale cache file /home/peltzer/.julia/compiled/v1.0/CRCBS/26Ksw.ji for CRCBS [edef75b8-3ac4-11e9-0392-bf149c536ee3]
└ @ Base loading.jl:1190
┌ Warning: Replacing docs for `CRCBS.get_dist_matrix :: Tuple{LightGraphs.AbstractGraph}` in module `CRCBS`
└ @ Base.Docs docs/Docs.jl:223
┌ Warning: Replacing docs for `CRCBS.pad_matrix :: Union{Tuple{T}, Tuple{Array{T,2},Tuple{Int64,Int64},T}} where T` in module `CRCBS`
└ @ Base.Docs docs/Docs.jl:223
┌ Warning: Replacing docs for `CRCBS.initialize_full_grid_graph :: Tuple{}` in module `CRCBS`
└ @ Base.Docs docs/Docs.jl:223
┌ Warning: Replacing docs for `CRCBS.initialize_regular_grid_graph :: Tuple{}` in module `CRCBS`
└ @ Base.Docs docs/Docs.jl:223


MAPF{MetaDiGraph{Int64,Float64}}({121, 440} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0), [1, 5, 8], [4, 1, 2], 0.5, 0.1, 0.5)

In [2]:
CTCBS(mapf)

-------------empty path sent --------------------Concerned node: 2
-------------empty path sent --------------------Concerned node: 12
-------------empty path sent --------------------Concerned node: 4
-------------empty path sent --------------------Concerned node: 6
-------------empty path sent --------------------Concerned node: 16
-------------empty path sent --------------------Concerned node: 7
-------------empty path sent --------------------Concerned node: 9
-------------empty path sent --------------------Concerned node: 19

 
Trying to get the most likely conflict from solution: 
Array{LightGraphs.SimpleGraphs.SimpleEdge,1}[[Edge 1 => 2, Edge 2 => 3, Edge 3 => 4], [Edge 5 => 4, Edge 4 => 3, Edge 3 => 2, Edge 2 => 1], [Edge 8 => 7, Edge 7 => 6, Edge 6 => 5, Edge 5 => 4, Edge 4 => 3, Edge 3 => 2]]
occupancy test 1 
Dict(1=>(2, 1.0))
occupancy test 2
Dict{Int64,Tuple{Int64,Float64}}()
 
Found edge conflict: 
Agents: 1 and 2
Edge: 2 => 3
Collision Probability: 0.6016454427090591


(Array{LightGraphs.SimpleGraphs.SimpleEdge,1}[[Edge 1 => 12, Edge 12 => 13, Edge 13 => 14, Edge 14 => 15, Edge 15 => 4], [Edge 5 => 4, Edge 4 => 3, Edge 3 => 2, Edge 2 => 1], [Edge 8 => 7, Edge 7 => 6, Edge 6 => 5, Edge 5 => 4, Edge 4 => 3, Edge 3 => 2]], 15)

In [1]:
mg = MetaDiGraph()
add_vertex!(mg,Dict(:x=>1,:n_delay=>1.0))
add_vertex!(mg,Dict(:x=>2,:n_delay=>1.0))
add_edge!(mg,1,2)
set_prop!(mg,Edge(1,2),:foo,5)

UndefVarError: UndefVarError: MetaDiGraph not defined

In [5]:
for v in vertices(mg)
    print(v)
end

12

In [6]:
Edge(1,2).src

1

In [12]:
get_prop(mg,Edge(2,1),:foo)

5

In [3]:
using CBS

ArgumentError: ArgumentError: Package CBS not found in current path:
- Run `import Pkg; Pkg.add("CBS")` to install the CBS package.
